In [442]:
! ls /eos/uscms/store/user/cmantill/boostedhiggs/Jul21_2017/

DYJetsToLL_M-10to50	       SingleMuon_Run2017B
DYJetsToLL_M-50_HT-100to200    SingleMuon_Run2017C
DYJetsToLL_M-50_HT-1200to2500  SingleMuon_Run2017D
DYJetsToLL_M-50_HT-200to400    SingleMuon_Run2017E
DYJetsToLL_M-50_HT-2500toInf   SingleMuon_Run2017F
DYJetsToLL_M-50_HT-400to600    ST_s-channel_4f_hadronicDecays
DYJetsToLL_M-50_HT-600to800    ST_s-channel_4f_leptonDecays
DYJetsToLL_M-50_HT-70to100     ST_t-channel_antitop_4f_InclusiveDecays
DYJetsToLL_M-50_HT-800to1200   ST_t-channel_top_4f_InclusiveDecays
DYJetsToLL_Pt-100To250	       ST_tW_antitop_5f_inclusiveDecays
DYJetsToLL_Pt-250To400	       ST_tW_top_5f_inclusiveDecays
DYJetsToLL_Pt-400To650	       ttHToNonbb_M125
DYJetsToLL_Pt-50To100	       TTTo2L2Nu
DYJetsToLL_Pt-650ToInf	       TTToHadronic
GluGluHToTauTau		       TTToSemiLeptonic
GluGluHToWW_Pt-200ToInf_M-125  VBFHToWWToLNuQQ_M-125_withDipoleRecoil
GluGluZH_HToWW_ZTo2L_M-125     WJetsToLNu_HT-100To200
HWminusJ_HToWW_M-125	       WJetsToLNu_HT-1200To2500
HWplusJ_HToWW_M-125	 

In [417]:
import pandas as pd
path = "/eos/uscms/store/user/cmantill/boostedhiggs/Jul21_2017/GluGluHToWW_Pt-200ToInf_M-125/outfiles/0-1_ele.parquet"
df = pd.read_parquet(path)

for key in df.keys():
    if "weight" in key and "Up" in key and "mu" in key:
        print(key)

weight_mu_id_muonUp
weight_mu_L1PrefiringUp
weight_mu_trigger_noniso_muonUp
weight_mu_UEPS_ISRUp
weight_mu_btagSFbc_correlatedUp
weight_mu_btagSFlight_2017Up
weight_mu_PDF_weightUp
weight_mu_scalevar_7ptUp
weight_mu_aS_weightUp
weight_mu_UEPS_FSRUp
weight_mu_pileupUp
weight_mu_isolation_muonUp
weight_mu_btagSFbc_2017Up
weight_mu_trigger_iso_muonUp
weight_mu_scalevar_3ptUp
weight_mu_btagSFlight_correlatedUp
weight_mu_PDFaS_weightUp


# Make datacard


In [398]:
from typing import Dict, List, Tuple, Union

import json
import logging
from collections import OrderedDict

In [399]:
from __future__ import print_function, division
import sys
import os
import rhalphalib as rl
import numpy as np
import scipy.stats
rl.util.install_roofit_helpers()
rl.ParametericSample.PreferRooParametricHist = False

In [400]:
CMS_PARAMS_LABEL = "CMS_HWW_boosted"

In [401]:
year = "2017"   # can give "all"
lep_ch = "mu"

with open("../fileset/luminosity.json") as f:
    LUMI = json.load(f)[lep_ch]

if year != "all":
    years = [year]
    full_lumi = LUMI[year]
else:
    years = ["2016", "2016APV", "2017", "2018"]
    full_lumi = np.sum(list(LUMI.values()))

full_lumi

41475.26

In [418]:
# systematics that are NOT necessarily stored in the parquets
sys_dict = {
    f"lumi_13TeV_{year}": rl.NuisanceParameter(f'CMS_lumi_13TeV_{year}', 'lnN'),
    "BR_hww": rl.NuisanceParameter(f'BR_hww', 'lnN'),
}

# tuple (value_up, value_down) and if (value_up, None) is given then value_down=value_up
sys_dict_values = {
    f"lumi_13TeV_{year}": (1.02 ** (LUMI["2017"] / full_lumi), None),
    "BR_hww": (1.0153, 0.9848)
}

# systematics that ARE stored in the parquets
sys_from_parquets = {
    "mu": {
        "all_samples": {
            "weight_mu_btagSFlight_2017": rl.NuisanceParameter(f'{CMS_PARAMS_LABEL}_btagSFlight_{year}', 'lnN'),
            "weight_mu_btagSFlight_correlated": rl.NuisanceParameter(f'{CMS_PARAMS_LABEL}_btagSFlight_correlated', 'lnN'),
            "weight_mu_btagSFbc_2017": rl.NuisanceParameter(f'{CMS_PARAMS_LABEL}_btagSFbc_{year}', 'lnN'),
            "weight_mu_btagSFbc_correlated": rl.NuisanceParameter(f'{CMS_PARAMS_LABEL}_btagSFbc_correlated', 'lnN'),
            "weight_mu_pileup": rl.NuisanceParameter(f'{CMS_PARAMS_LABEL}_PU_{year}', 'shape'),
            "weight_mu_isolation_muon": rl.NuisanceParameter(f'CMS_mu_iso_{year}', 'lnN'),
            "weight_mu_id_muon": rl.NuisanceParameter(f'CMS_mu_id_{year}', 'lnN'),
            "weight_mu_L1Prefiring": rl.NuisanceParameter(f'CMS_L1Prefiring_{year}', 'lnN'),
            "weight_mu_trigger_iso_muon": rl.NuisanceParameter('CMS_btagSF', 'lnN'),
            "weight_mu_trigger_noniso_muon": rl.NuisanceParameter(f'{CMS_PARAMS_LABEL}_mu_trigger_{year}', 'lnN'),
        },
        ### signal
        "HWW": {
            "weight_mu_UEPS_FSR": rl.NuisanceParameter('UEPS_FSR_ggF', 'shape'),
            "weight_mu_UEPS_ISR": rl.NuisanceParameter('UEPS_ISR_ggF', 'shape'),
            "weight_mu_PDF_weight": rl.NuisanceParameter('pdf_Higgs_ggF', 'lnN'),
            "weight_mu_PDFaS_weight": rl.NuisanceParameter('pdfAS_Higgs_ggF', 'lnN'),
            "weight_mu_aS_weight": rl.NuisanceParameter('aS_Higgs_ggF', 'lnN'),                        
            "weight_mu_scalevar_3pt": rl.NuisanceParameter(f'{CMS_PARAMS_LABEL}_scale_pt_3_ggF_{year}', 'lnN'),
            "weight_mu_scalevar_7pt": rl.NuisanceParameter(f'{CMS_PARAMS_LABEL}_scale_pt_7_ggF_{year}', 'lnN'),
        },
        "VBF": {
            "weight_mu_UEPS_FSR": rl.NuisanceParameter('UEPS_FSR_VBF', 'shape'),
            "weight_mu_UEPS_ISR": rl.NuisanceParameter('UEPS_ISR_VBF', 'shape'),
            "weight_mu_PDF_weight": rl.NuisanceParameter('pdf_Higgs_VBF', 'lnN'),
            "weight_mu_PDFaS_weight": rl.NuisanceParameter('pdfAS_Higgs_VBF', 'lnN'),
            "weight_mu_aS_weight": rl.NuisanceParameter('aS_Higgs_VBF', 'lnN'),                        
            "weight_mu_scalevar_3pt": rl.NuisanceParameter(f'{CMS_PARAMS_LABEL}_scale_pt_3_VBF_{year}', 'lnN'),
            "weight_mu_scalevar_7pt": rl.NuisanceParameter(f'{CMS_PARAMS_LABEL}_scale_pt_7_VBF_{year}', 'lnN'),            
        },
        "VH": {},
        "ttH": {},
        ### bkgs        
        "TTbar": {},
        "WJetsLNu": {
            "weight_mu_d1kappa_EW": rl.NuisanceParameter(f'{CMS_PARAMS_LABEL}_W_d1kappa_EW_{year}', 'lnN'),
            "weight_mu_W_d2kappa_EW": rl.NuisanceParameter(f'{CMS_PARAMS_LABEL}_W_d2kappa_EW_{year}', 'lnN'),            
            "weight_mu_W_d3kappa_EW": rl.NuisanceParameter(f'{CMS_PARAMS_LABEL}_W_d3kappa_EW_{year}', 'lnN'),            
            "weight_mu_d1K_NLO": rl.NuisanceParameter(f'{CMS_PARAMS_LABEL}_d1K_NLO_{year}', 'lnN'),            
            "weight_mu_d2K_NLO": rl.NuisanceParameter(f'{CMS_PARAMS_LABEL}_d2K_NLO_{year}', 'lnN'),            
            "weight_mu_d3K_NLO": rl.NuisanceParameter(f'{CMS_PARAMS_LABEL}_d3K_NLO_{year}', 'lnN'),            
        },
        "SingleTop": {},    
        "DYJets": {
            "weight_mu_d1kappa_EW": rl.NuisanceParameter(f'{CMS_PARAMS_LABEL}_Z_d1kappa_EW_{year}', 'lnN'),
            "weight_mu_Z_d2kappa_EW": rl.NuisanceParameter(f'{CMS_PARAMS_LABEL}_Z_d2kappa_EW_{year}', 'lnN'),            
            "weight_mu_Z_d3kappa_EW": rl.NuisanceParameter(f'{CMS_PARAMS_LABEL}_Z_d3kappa_EW_{year}', 'lnN'),                     
        },        
    },
}

In [419]:
def shape_to_num(var, nom, clip=1.5):
    nom_rate = np.sum(nom)
    var_rate = np.sum(var)

    if abs(var_rate/nom_rate) > clip:
        var_rate = clip*nom_rate

    if var_rate < 0:
        var_rate = 0

    return var_rate/nom_rate

def get_template(region, sample, ptbin):
    return (
        hists_templates[region][{"samples": sample, "systematic": "Nominal", "fj_pt": ptbin}].values(),
        massbins,
        "reco_higgs_m"
    )

# Load template

In [433]:
import pickle as pkl
with open("hists_templates.pkl", "rb") as f:
    hists_templates = pkl.load(f)

In [434]:
hists_templates

{'pass': Hist(
   StrCategory(['WJetsLNu', 'DYJets', 'VBF', 'VH', 'TTbar', 'SingleTop', 'HWW', 'WZQQ', 'Data', 'QCD', 'Diboson', 'ttH'], growth=True, name='samples'),
   StrCategory(['Nominal', 'weight_ele_btagSFlight_2017Up', 'weight_ele_btagSFlight_2017Down', 'weight_ele_btagSFlight_correlatedUp', 'weight_ele_btagSFlight_correlatedDown', 'weight_ele_btagSFbc_2017Up', 'weight_ele_btagSFbc_2017Down', 'weight_ele_btagSFbc_correlatedUp', 'weight_ele_btagSFbc_correlatedDown', 'weight_ele_pileupUp', 'weight_ele_pileupDown', 'weight_ele_isolation_electronUp', 'weight_ele_isolation_electronDown', 'weight_ele_id_electronUp', 'weight_ele_id_electronDown', 'weight_ele_L1PrefiringUp', 'weight_ele_L1PrefiringDown', 'weight_ele_trigger_electronUp', 'weight_ele_trigger_electronDown', 'weight_ele_reco_electronUp', 'weight_ele_reco_electronDown', 'weight_ele_d1K_NLOUp', 'weight_ele_d1K_NLODown', 'weight_ele_d2K_NLOUp', 'weight_ele_d2K_NLODown', 'weight_ele_d3K_NLOUp', 'weight_ele_d3K_NLODown', 'weigh

In [435]:
samples = list(hists_templates["pass"].axes[0])
samples

['WJetsLNu',
 'DYJets',
 'VBF',
 'VH',
 'TTbar',
 'SingleTop',
 'HWW',
 'WZQQ',
 'Data',
 'QCD',
 'Diboson',
 'ttH']

# Create datacard

In [436]:
# (name in templates, name in cards)
labels = {
    # sigs
    "HWW": "ggF",
    "VBF": "VBF",
    "VH": "VH",
    "ttH": "ttH",
    # BKGS
    "QCD": "qcd",
    "TTbar": "ttbar",
    "WJetsLNu": "wjets",
    "SingleTop": "singletop",
    "DYJets": "zjets",
}

bkgs = [
    "TTbar", 
    "WJetsLNu", 
    "SingleTop", 
    "DYJets"
]

sigs = [
    "HWW", 
    "VBF", 
    "VH", 
    "ttH"
]
samples = sigs+bkgs


samples

['HWW', 'VBF', 'VH', 'ttH', 'TTbar', 'WJetsLNu', 'SingleTop', 'DYJets']

In [437]:
ptbins = hists_templates["pass"].axes[2].edges
npt = len(ptbins) - 1

massbins = hists_templates["pass"].axes[3].edges
mass = rl.Observable('reco_higgs_m', massbins)

# here we derive these all at once with 2D array
ptpts, masspts = np.meshgrid(ptbins[:-1] + 0.3 * np.diff(ptbins), massbins[:-1] + 0.5 * np.diff(massbins), indexing='ij')

rhopts = 2*np.log(masspts/ptpts)
pt_scaled = (ptpts - ptbins[0]) / (ptbins[-1] - ptbins[0])
rho_scaled = (rhopts - (-6)) / ((-2.1) - (-6))

validbins = (rho_scaled >= 0) & (rho_scaled <= 1)
rho_scaled[~validbins] = 1  # we will mask these out later

### Raghav: https://github.com/rkansal47/HHbbVV/blob/68dd5738ebe6950a6b5ea16049c4047b7de7892d/src/HHbbVV/postprocessing/CreateDatacard.py#L66-L77
# bins = hists_templates["pass"][{"samples": sum, "rec_higgs_m": sum, "systematic": sum}].values()
# pts = bins[:-1] + 0.5 * np.diff(bins)
# ptscaled = (pts - ptbins[0]) / (ptbins[-1] - ptbins[0])

In [438]:
# build actual fit model now
model = rl.Model("testModel")

In [439]:
# fill datacard with systematics and rates
for ptbin in range(npt):
    for region in ['pass', 'fail']:
        ch = rl.Channel("ptbin%d%s" % (ptbin, region))
        model.addChannel(ch)

        isPass = region == 'pass'
        ptnorm = 1.

        for sName in samples:

            templ = get_template(region, sName, ptbin)
            stype = rl.Sample.SIGNAL if sName in sigs else rl.Sample.BACKGROUND
            sample = rl.TemplateSample(ch.name + '_' + labels[sName], stype, templ)

            ### systematics NOT from parquets
            for sys_name, sys_value in sys_dict.items():
                sample.setParamEffect(sys_value, 
                                      sys_dict_values[sys_name][0], 
                                      sys_dict_values[sys_name][1] if sys_dict_values[sys_name][1] is not None else sys_dict_values[sys_name][0])
            

            ### systematics from parquets
            # apply systematics that are common for all samples
            for sys_name, sys_value in sys_from_parquets[lep_ch]["all_samples"].items():
                syst_up = hists_templates[region][{"samples": sName, "fj_pt": ptbin, "systematic": sys_name+"Up"}].values()
                syst_do = hists_templates[region][{"samples": sName, "fj_pt": ptbin, "systematic": sys_name+"Down"}].values()
                nominal = hists_templates[region][{"samples": sName, "fj_pt": ptbin, "systematic": "Nominal"}].values()
                
                eff_up = shape_to_num(syst_up,nominal)
                eff_do = shape_to_num(syst_do,nominal)
                
                sample.setParamEffect(sys_value, eff_up, eff_do)

            # apply systematics that are common for this particular sample
            for sys_name, sys_value in sys_from_parquets[lep_ch][sName].items():
                syst_up = hists_templates[region][{"samples": sName, "fj_pt": ptbin, "systematic": sys_name+"Up"}].values()
                syst_do = hists_templates[region][{"samples": sName, "fj_pt": ptbin, "systematic": sys_name+"Down"}].values()
                nominal = hists_templates[region][{"samples": sName, "fj_pt": ptbin, "systematic": "Nominal"}].values()
                
                eff_up = shape_to_num(syst_up,nominal)
                eff_do = shape_to_num(syst_do,nominal)
                
                sample.setParamEffect(sys_value, eff_up, eff_do)

            ch.addSample(sample)
        
        # add data
        data_obs = get_template(region, "Data", ptbin)
        ch.setObservation(data_obs)       

effect_down (CMS_HWW_boosted_PU_2017, ptbin0pass_ttH) has magnitude greater than 50% (54.76%), you might be passing absolute values instead of relative
effect_down (CMS_HWW_boosted_PU_2017, ptbin0pass_zjets) has magnitude greater than 50% (52.96%), you might be passing absolute values instead of relative
effect_down (CMS_HWW_boosted_PU_2017, ptbin0fail_ttH) has magnitude greater than 50% (54.11%), you might be passing absolute values instead of relative
effect_down (CMS_HWW_boosted_PU_2017, ptbin0fail_ttbar) has magnitude greater than 50% (50.09%), you might be passing absolute values instead of relative
effect_down (CMS_HWW_boosted_PU_2017, ptbin0fail_zjets) has magnitude greater than 50% (56.03%), you might be passing absolute values instead of relative
effect_down (CMS_HWW_boosted_PU_2017, ptbin1pass_ttH) has magnitude greater than 50% (50.37%), you might be passing absolute values instead of relative
effect_up (CMS_HWW_boosted_PU_2017, ptbin1pass_zjets) has magnitude greater than 5

/tmp/ipykernel_26385/4285621970.py:5: RuntimeWarning: invalid value encountered in double_scalars
  if abs(var_rate/nom_rate) > clip:
/tmp/ipykernel_26385/4285621970.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return var_rate/nom_rate


In [440]:
# qcd data-driven estimation

# get the transfer factor
qcd_eff = (
    hists_templates["pass"][{"samples": "QCD", "systematic": "Nominal"}].sum()
    / hists_templates["fail"][{"samples": "QCD", "systematic": "Nominal"}].sum()
)

tf_dataResidual = rl.BernsteinPoly(
    f"{CMS_PARAMS_LABEL}_tf_dataResidual", (2, 2), ['pt', 'rho'], limits=(-20, 20)
)
tf_dataResidual_params = tf_dataResidual(pt_scaled, rho_scaled)
tf_params_pass = qcd_eff * tf_dataResidual_params

for ptbin in range(npt):
    failCh = model['ptbin%dfail' % ptbin]
    passCh = model['ptbin%dpass' % ptbin]

    qcdparams = np.array([rl.IndependentParameter('qcdparam_ptbin%d_massbin%d' % (ptbin, i), 0) for i in range(mass.nbins)])
    initial_qcd = failCh.getObservation().astype(float)  # was integer, and numpy complained about subtracting float from it
    
    for sample in failCh:
        initial_qcd -= sample.getExpectation(nominal=True)
    
    if np.any(initial_qcd < 0.):
        initial_qcd[np.where(initial_qcd<0)] = 0
#         raise ValueError("initial_qcd negative for some bins..", initial_qcd)
    
    sigmascale = 10  # to scale the deviation from initial
    scaledparams = initial_qcd * (1 + sigmascale/np.maximum(1., np.sqrt(initial_qcd)))**qcdparams
    fail_qcd = rl.ParametericSample('ptbin%dfail_qcd' % ptbin, rl.Sample.BACKGROUND, mass, scaledparams)
    failCh.addSample(fail_qcd)
    pass_qcd = rl.TransferFactorSample('ptbin%dpass_qcd' % ptbin, rl.Sample.BACKGROUND, tf_params_pass[ptbin, :], fail_qcd)
    passCh.addSample(pass_qcd) 

In [441]:
tmpdir = "tmpdir/"

if not os.path.isdir(tmpdir):
    os.mkdir(tmpdir)

with open(f"{tmpdir}/testModel.pkl", "wb") as fout:
    pkl.dump(model, fout, protocol=2)

In [144]:
# # you can then render combine in a CMSENV

# with open(os.path.join(str(tmpdir), 'testModel.pkl'), "rb") as fout:
#     model = pickle.load(fout)

# model.renderCombine(os.path.join(str(tmpdir), 'testModel'))       